In [1]:
import numpy as np
import matplotlib.pyplot as plt
import meep

# Direct solution of maxwell equations

In [2]:
"""
This example simulates a spherical point source inside a cube with reflecting walls.
A continuous (fixed frequency) point source (implemented as a zero–sized source) is placed at the center
of a 16×16×16 cube (in arbitrary units). No PML layers are added so that all cell boundaries are fully reflective.
After running the simulation, a 2D slice (at z = 0) of the electric field component Ez is extracted and plotted.
"""

import meep as mp
import matplotlib.pyplot as plt
import numpy as np

# -----------------------------
# Simulation Setup
# -----------------------------
resolution = 20                 # pixels per unit length
cell_size = mp.Vector3(16, 16, 16)  # Define a cube with side length 16

# Define source frequency (in Meep units; here, frequency=1.0 corresponds to a vacuum wavelength of 1)
f_source = 1.0

# Define a point source at the center of the cell.
# In Meep a point source is created by setting the "size" to zero.
source = mp.Source(
    src=mp.ContinuousSource(frequency=f_source),
    component=mp.Ez,                   # We choose the Ez component. (For a true dipole, one might use a vector source.)
    center=mp.Vector3(0, 0, 0),         # Place at the center of the cube.
    size=mp.Vector3(0, 0, 0)            # Zero size gives a point source (spherical radiation).
)

# Set up the simulation.
# By not including any PML layers (i.e. boundary_layers is an empty list),
# the simulation domain uses reflecting boundary conditions (akin to PEC walls).
sim = mp.Simulation(
    cell_size=cell_size,
    sources=[source],
    resolution=resolution,
    default_material=mp.Medium(index=1.0),  # Set the medium to vacuum (dielectric constant 1)
    boundary_layers=[]                      # No absorbing layers → reflecting (metallic) boundaries.
)

# -----------------------------
# Running the Simulation
# -----------------------------
# Run the simulation for a total time of 50 time units.
sim.run(until=50)

# -----------------------------
# Extracting and Visualizing Field Data
# -----------------------------
# Extract a 2D slice of the Ez field at the mid-plane z = 0.
# Setting the size vector's z-component to 0 instructs get_array() to extract a slice.
ez_slice = sim.get_array(center=mp.Vector3(0, 0, 0),
                         size=mp.Vector3(16, 16, 0),
                         component=mp.Ez)

# Create a plot of the slice.
plt.figure(figsize=(6, 6))
# The extent is chosen so that the x,y coordinates range from -8 to 8 (centering the cell).
plt.imshow(ez_slice, interpolation='spline16', cmap='RdBu', extent=[-8, 8, -8, 8])
plt.xlabel("x")
plt.ylabel("y")
plt.title("Ez field (z=0 plane) at t = 50")
plt.colorbar(label="Ez amplitude")
plt.show()